In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Reshape
from keras import layers
import matplotlib.pyplot as plt
import os 
import cv2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [3]:
x=[]
y=[]
mapping={
    "anger":0,
    "disgust":1,
    "fear":2,
    "happy":3,
    "neutral":4
}
path=r"C:\Users\Suma\Desktop\FACE_RECOGNITION\Emotion"
for relate in mapping.keys():
    for img in os.listdir(os.path.join(path,relate)):
        im=cv2.imread(os.path.join(path,relate,img))
        im=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im=cv2.resize(im,(40,40))
        x.append(im)
        y.append(mapping[relate])
x=np.array(x)
y=np.array(y)
x=x.reshape(x.shape[0],40,40,1)
x=x.astype('float32')
x=x/255
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
x_train,x_valid,y_train,y_valid=train_test_split(x_train,y_train,test_size=0.1)
early_stopping=EarlyStopping(
    patience=25, 
    restore_best_weights=True,
)

In [4]:
x_valid.shape

(344, 40, 40, 1)

In [5]:
emotion =Sequential()
emotion.add(Conv2D(filters=25,kernel_size=3,strides=(1,1),padding="valid",input_shape=(40,40,1)))
emotion.add(layers.LeakyReLU(alpha=0.1))
emotion.add(MaxPool2D(pool_size=(2,2)))
emotion.add(layers.BatchNormalization()) 
emotion.add(Conv2D(filters=10,kernel_size=5,strides=(1,1),padding="same")) 
emotion.add(layers.LeakyReLU(alpha=0.1))
emotion.add(MaxPool2D(pool_size=(2,2)))
emotion.add(layers.BatchNormalization())
emotion.add(Conv2D(filters=5,kernel_size=7,strides=(1,1),padding="valid")) 
emotion.add(layers.LeakyReLU(alpha=0.1))
emotion.add(MaxPool2D(pool_size=(2,2)))
emotion.add(layers.BatchNormalization())
emotion.add(Flatten())
emotion.add(Dense(5,activation="softmax"))
emotion.summary()
emotion.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
emotion.fit(x_train,y_train,epochs=200,callbacks=[early_stopping],validation_data=(x_valid, y_valid)) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 38, 25)        250       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 38, 38, 25)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 19, 25)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 19, 19, 25)        100       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 10)        6260      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 19, 19, 10)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 10)          0

In [6]:
print(emotion.evaluate(x_test,y_test))

12/12 [==============================] - 0s 22ms/step - loss: 1.1524 - accuracy: 0.5340
[1.1524022817611694, 0.5340313911437988]


In [7]:
emotion.save("emotion_model.h5")

In [8]:
new_model=keras.models.load_model("emotion_model.h5")

In [9]:
new_model.predict(x_valid[0].reshape(-1,40,40,1))

array([[0.06600445, 0.02694964, 0.03026776, 0.00996546, 0.86681265]],
      dtype=float32)